In [1]:
import datetime
import locale

#from django.contrib import messages
#from django.utils.translation import gettext as _
import pandas as pd
import plotly.graph_objects as go
import dash  
from dash import (
    dcc,
    html,
    Input,
    Output,
    State,
)
#from django_plotly_dash import DjangoDash
from wetterdienst import Resolution, Period
from wetterdienst.provider.dwd.observation import DwdObservationRequest, DwdObservationDataset
from typing import Tuple
from plotly.subplots import make_subplots
from dash.exceptions import PreventUpdate


#from .Warmelastapproximation_csv import heatLoad

In [2]:
pip list

Package                           Version
--------------------------------- ------------
aenum                             3.1.15
aiobotocore                       2.7.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.12
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.1.4
anaconda-navigator                2.5.2
anaconda-project                  0.11.1
annotated-types                   0.7.0
anyio                             4.2.0
appdirs                           1.4.4
archspec                          0.2.1
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           5.3.4
asttokens               

In [3]:
import logging
from datetime import datetime
logging.basicConfig(level=logging.DEBUG)
#print(datetime.now())
#logging.debug(f"Received input value: {random.seed}")

In [4]:

import math
import pathlib
import os.path 
import pandas as pd
from typing import Tuple
import datetime 
from wetterdienst.provider.dwd.observation import DwdObservationRequest
import os
PATH = os.getcwd() 
#PATH = pathlib.Path(__file__).parent.resolve() 
DATA_PATH = os.path.join(PATH , 'Wärme_Strom.csv') 
TRY_PATH = os.path.join(PATH , 'TRY2015_524124130664_Jahr.csv')


def heatLoad(application:int,heatDemand:int,station:int,startDate:str,endDate:str,
             referenceYear:str)-> Tuple[int, pd.DataFrame,pd.DataFrame]:
    logging.debug(f"Received input value: {referenceYear}")
    if referenceYear == "off":
        # ToDo check with Firas why this parameter is "on", when 
        #Setting up the resolution for data filtering
        RESOLUTION = 'HOURLY'
        # Parameter variable selection
        PARAMETER = 'TEMPERATURE_AIR_MEAN_200'
        #Setting up the Period
        PERIOD = 'RECENT'
        # Acquiring all the stations that provide data according to selected filters
        stations = DwdObservationRequest(
            parameter = PARAMETER,
            resolution = RESOLUTION,
            period = PERIOD
            )
        #station = int(station)
        data = stations.filter_by_station_id(station_id = station)
        stationData = data.values.all().df.to_pandas()

        #Input from the server is Im Kelvin converted to celsius
        stationData['value'] = stationData['value'].apply(lambda x: x - 273.15)
    else:
        stationData = pd.read_csv(TRY_PATH)
        stationData["date"] = pd.to_datetime(stationData[[  "YEAR","MONTH", 
                                                            "DAY", "HOUR"]],
                                                            format = "%Y-%m-%d %H",
                                                            utc = True)
    # Fill in the missing entries in the wetterdient data and mark them
    missingValues=0
    stationData['fehlend'] = 'False'
    for i in range (0,len(stationData.index)):
        if math.isnan(stationData['value'][i]):
            missingValues += 1
            stationData['value'][i] = stationData['value'][i-24]
            stationData['fehlend'][i] = 'True'
    # Prepare csv file to read factors
    dfHeat = pd.read_csv(DATA_PATH)

    #Load regression coefficients
    A = float(dfHeat.iloc[:,9][application+4])
    B = float(dfHeat.iloc[:,10][application+4])
    C = float(dfHeat.iloc[:,11][application+4])
    D = float(dfHeat.iloc[:,12][application+4])

    #Load weekday factors
    weekDay = []
    ##Montag
    weekDay.append(float(dfHeat.iloc[:,9][application+23]))
    ##Dienstag
    weekDay.append(float(dfHeat.iloc[:,10][application+23]))
    ## Mittwoch
    weekDay.append(float(dfHeat.iloc[:,11][application+23]))
    ##Donnerstag
    weekDay.append(float(dfHeat.iloc[:,12][application+23]))
    ##Freitag
    weekDay.append(float(dfHeat.iloc[:,13][application+23]))
    ##Samstag
    weekDay.append(float(dfHeat.iloc[:,14][application+23]))
    ##Sonntag
    weekDay.append(float(dfHeat.iloc[:,15][application+23]))

    #Calculation of h
    h = []
    for i in range(0,stationData.shape[0]):
        tAverage = 0
        runV = i 
        j = 0
   
        while j<24 and runV <stationData.shape[0] :
            tMoment = float(stationData['value'][runV])
            tAverage = tAverage+tMoment/24
            runV = runV + 1
            j += 1
  
        h.append(round( A / (1 + (B / (tAverage - 40)) ** C) + D,14))

    #Load hour factors
    lineStart = application*13 -24

    #Load hour factors for Mondays
    column = 18
    factorMonday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorMonday[j][i] = dfHeat.iloc[:,column+i][lineStart+j]

    #Load hour factors for Tuesdays
    column = 44
    factorTuesday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorTuesday[j][i] = dfHeat.iloc[:,column+i][lineStart+j]

    #Load hour factors for Wednesdays
    column = 70
    factorWednesday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorWednesday[j][i] = dfHeat.iloc[:,column+i][lineStart+j]

    #Load hour factors for Thursdays
    column = 96
    factorThursday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorThursday[j][i] = dfHeat.iloc[:,column+i][lineStart+j]

    #Load hour factors for Fridays
    column = 122
    factorFriday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorFriday[j][i] = dfHeat.iloc[:,column+i][lineStart+j]


    #Load hour factors for Saturdays
    column = 148
    factorSaturday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorSaturday[j][i]=dfHeat.iloc[:,column+i][lineStart+j]

    #Load hour factors for Sundays
    column = 174
    factorSunday = [[0 for x in range(24)] for y in range(10)] 
    for i in range(0,24):
        for j in range(0,10):
            factorSunday[j][i] = dfHeat.iloc[:,column+i][lineStart+j]

    # Calculation of the hourly heat demand
    Q_average = heatDemand
    Q = []

    # Calculate mean h
    h_average = 0
    for i in range(0,stationData.shape[0]):
        h_average = h_average+h[i]

    h_average = h_average / (stationData.shape[0])

    i=0
    for k in range (0,stationData.shape[0]):
                # Reset hours after 24 hours
                if i == 24 :
                    i = 0
                #Selection of the line depending on the outside temperature
                if stationData['value'][k] <=-15:
                    line = 0
                elif stationData['value'][k] <=-10 and stationData['value'][k] >-15:
                    line = 1
                elif stationData['value'][k] <=-5 and stationData['value'][k] >-10:
                    line = 2
                elif stationData['value'][k] <=0 and stationData['value'][k] >-5:
                    line = 3
                elif stationData['value'][k] <=5 and stationData['value'][k] >0:
                    line = 4
                elif stationData['value'][k] <=10 and stationData['value'][k] >5:
                    line = 5
                elif stationData['value'][k] <=15 and stationData['value'][k] >10:
                    line = 6
                elif stationData['value'][k] <=20 and stationData['value'][k] >15:
                    line = 7
                    #Here is a difference
                elif stationData['value'][k] <=25 and stationData['value'][k] >20:
                    line = 8
                else:
                    line = 9
                #Tagessumme h
                h_sum = 0
                runV = k 
                j = 0
                while j<24 and runV<stationData.shape[0]:
                    h_sum = h_sum + h[runV]
                    runV = runV+1
                    j += 1
                #Selecting the right hour factor
                if stationData['date'][k].weekday() == 0 :
                    F=factorMonday[line] [i]
                elif stationData['date'][k].weekday() == 1 :
                    F = factorTuesday[line] [i]
                elif stationData['date'][k].weekday() == 2 :
                    F = factorWednesday[line] [i]
                elif stationData['date'][k].weekday() == 3:
                    F = factorThursday[line] [i]
                elif stationData['date'][k].weekday() == 4:
                    F = factorFriday[line] [i]
                elif stationData['date'][k].weekday() == 5 :
                    F = factorSaturday[line] [i]
                else:
                    F = factorSunday[line] [i]
                try:
                   
                    Q.append( h_sum * (Q_average / h_average) * F * 
                             weekDay[stationData['date'][k].weekday()])
                except ZeroDivisionError:
                    pass
                #Increment hours
                i = i+1
                
    #Calculation of the approximate annual heat requirement
    qSum = 0
    for i in range (0,stationData.shape[0]):
        qSum = qSum+Q[i]
        

    # Correction of the heat requirement, calculation of the WW_share
    qWW = []
    Q_INPUT = heatDemand
    for i in range(0,stationData.shape[0]):
        Q[i] = Q[i]*(Q_INPUT/qSum)
        qWW.append(D*(Q[i]/h[i]))

    # start = pd.Timestamp(startDate + " 01:00:00+00:00")
    # end = pd.Timestamp(endDate + " 23:00:00+00:00")
    # # print(stationData.index[stationData.date == pd.Timestamp(startDate+" 01:00:00+00:00")].tolist())
    # start=stationData.index[stationData.date == pd.Timestamp(startDate+" 01:00:00+00:00")].tolist()[0]
    # end=stationData.index[stationData.date == pd.Timestamp(endDate+" 23:00:00+00:00")].tolist()[0] +1
   
    print("This ist the end 2", startDate, endDate)
    start=stationData[stationData.date == pd.Timestamp(startDate+" 01:00:00+00:00")].index.tolist()[0]
    end=stationData[stationData.date ==  pd.Timestamp(endDate+" 23:00:00+00:00")].index.tolist()[0] #+1
    heatApproximationDf = pd.DataFrame({'Time':stationData['date'][start:end],
                                        'Last':Q[start:end],
                                        'WW_Last':qWW[start:end],
                                        'fehlend':stationData['fehlend'][start:end]})
    return missingValues,heatApproximationDf


In [5]:
locale.setlocale(locale.LC_ALL, "de_DE.utf8") # German time

#app = DjangoDash('Warmelast')   
#Setting up the resolution for data filtering
resolution = 'HOURLY'

#Selecting the dataset

dataset = 'AIR_TEMPERATURE'
# Parameter variable selection

parameter = 'TEMPERATURE_AIR_MEAN_200'
#Setting up the Period
data = []
period = 'RECENT'
# Acquiring all the stations that provide data according to selected filters
stations = DwdObservationRequest(
    parameter = parameter,
    resolution = resolution,
    period = period
    )
placeholderState = "Auswahl des Bundesland"
print(stations.all().df.head())
try:
    polledStationNames = stations.all( ).df['state'].unique()
    print(polledStationNames)

except ValueError:
    polledStationNames = []
    placeholderState = "Weatherdata-API is not available. Please try again later."
    print(placeholderState)

INFO:wetterdienst.settings:Wetterdienst cache is enabled [CACHE_DIR: /home/trinker/.cache/wetterdienst]
INFO:wetterdienst.core.timeseries.request:Processing request DwdObservationRequest(parameter=[(tt_tu/air_temperature)], resolution=hourly, period=[recent], start_date=None, end_date=None, humanize=True, format=long, si_units=True)
DEBUG:asyncio:Using selector: EpollSelector
INFO:wetterdienst.util.network:Creating dircache folder at /home/trinker/.cache/wetterdienst/43200.0
INFO:wetterdienst.provider.dwd.observation.metaindex:Downloading file https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate//hourly/air_temperature/recent/TU_Stundenwerte_Beschreibung_Stationen.txt.
INFO:wetterdienst.util.network:Creating dircache folder at /home/trinker/.cache/wetterdienst-fsspec/None
INFO:wetterdienst.provider.dwd.observation.metaindex:Downloading file https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate//hourly/air_temperature/recent/TU_Stundenwerte

shape: (5, 8)
┌────────────┬───────────────┬──────────────┬──────────┬───────────┬────────┬──────────────┬───────┐
│ station_id ┆ start_date    ┆ end_date     ┆ latitude ┆ longitude ┆ height ┆ name         ┆ state │
│ ---        ┆ ---           ┆ ---          ┆ ---      ┆ ---       ┆ ---    ┆ ---          ┆ ---   │
│ str        ┆ datetime[μs,  ┆ datetime[μs, ┆ f64      ┆ f64       ┆ f64    ┆ str          ┆ str   │
│            ┆ UTC]          ┆ UTC]         ┆          ┆           ┆        ┆              ┆       │
╞════════════╪═══════════════╪══════════════╪══════════╪═══════════╪════════╪══════════════╪═══════╡
│ 00044      ┆ 2007-04-01    ┆ 2024-10-12   ┆ 52.9336  ┆ 8.237     ┆ 44.0   ┆ Großenkneten ┆ Frei  │
│            ┆ 00:00:00 UTC  ┆ 00:00:00 UTC ┆          ┆           ┆        ┆ Niedersachse ┆       │
│            ┆               ┆              ┆          ┆           ┆        ┆ n            ┆       │
│ 00073      ┆ 2007-04-01    ┆ 2024-10-12   ┆ 48.6183  ┆ 13.062    ┆ 374.0  ┆

In [6]:
'''stationId = "00044"
data=stations.filter_by_station_id(station_id=stationId)
stationData = data.values.all().df.to_pandas()
stationData'''

'stationId = "00044"\ndata=stations.filter_by_station_id(station_id=stationId)\nstationData = data.values.all().df.to_pandas()\nstationData'

In [7]:
'''minDate = (min(stationData['date'])).date()
maxDate = (max(stationData['date'])).date()
minDate'''

"minDate = (min(stationData['date'])).date()\nmaxDate = (max(stationData['date'])).date()\nminDate"

In [8]:

import datetime
import locale
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
from dash.dependencies import Input, Output
from wetterdienst.provider.dwd.observation import DwdObservationRequest
from typing import Tuple
from plotly.subplots import make_subplots

# Remove Django translation and use direct strings
def _(text):
    return text

locale.setlocale(locale.LC_ALL, "de_DE.utf8")  # German time

# Initialize a standard Dash app
app = Dash(__name__)

# Setting up the resolution for data filtering
resolution = 'HOURLY'
dataset = 'AIR_TEMPERATURE'
parameter = 'TEMPERATURE_AIR_MEAN_200'
period = 'RECENT'

# Acquiring all the stations that provide data according to selected filters
api_error = False
error_message = ""

stations = DwdObservationRequest(
    parameter=parameter,
    resolution=resolution,
    period=period
)
placeholderState = _("Auswahl des Bundesland")
try:
    polledStationNames = stations.all().df['state'].to_pandas().unique()
    print(polledStationNames)
    #dropdown_options = [{'label': name, 'value': value} for name, value in polledStationNames.to_list()]
    #print(dropdown_options)

except ValueError:
    polledStationNames = []
    api_error = True

    error_message = "Error accessing station data from Wetterdienst API. Please try again later."

# App layout
app.layout = html.Div([
    html.Div(id='api-error-message', children=[
        html.P(error_message)
    ], style={'display': 'block' if api_error else 'none', 'color': 'red', 'background': '#FFD2D2', 'padding': '10px'}),


    dcc.Store(id='on-load', data='loaded'),
    html.H1(_("Wärmelast Approximation"), style={'text-align': 'center'}, id="headingApp"),
    html.Div([
        html.P(_("Als Testrefenzjahr haben wir die folgenden Werte gewählt:"), id="container1"),
        html.P(_("Koordinatensystem : Lambert konform konisch"), id="container2"),
        html.P(_("Rechtswert        : 4201500 Meter"), id="container3"),
        html.P(_("Hochwert          : 2848500 Meter"), id="container4"),
        html.P(_("Höhenlage        : 36 Meter über NN"), id="container5"),
        html.P(_("Erstellung des Datensatzes im Mai 2016"), id="container6"),
        html.P(_("Art des TRY       : mittleres Jahr"), id="container7"),
        html.P(_("Bezugszeitraum    : 1995-2012"), id="container8"),
        html.P(_("Datenbasis        : Beobachtungsdaten Zeitraum 1995-2012"), id="container9"),
    ], id='hideText', style={'display': 'none'}),
    dcc.Dropdown(
        options=[
            {'label': _('Testreferenzjahr'), 'value': 'on'},
            {'label': _('Wetterstation'), 'value': 'off'}
        ],
        placeholder=_("Berechnungstyp"),
        id='referenceYear',
        value='on'
    ),
    html.Div([
        dcc.Dropdown(
            polledStationNames,
            placeholder=placeholderState,
            id='state'
        ),
        dcc.Dropdown(
            placeholder=_("Auswahl der Station"), 
            id='station'
        )
    ], id='hideElements', style={'display': 'block'}),

    dcc.Dropdown(
        options=[
            {'label': _('Einfamilienhaus'), 'value': '2'},
            {'label': _('Mehrfamilienhaus'), 'value': '3'},
            {'label': _('Gebietskörperschaft'), 'value': '4'},
            {'label': _('Einzelhandel, Großhandel'), 'value': '5'},
            {'label': _('Metall, Kfz'), 'value': '6'},
            {'label': _('sonst. betr. Dienstleistungen'), 'value': '7'},
            {'label': _('Gaststätten'), 'value': '8'},
            {'label': _('Beherbergung'), 'value': '9'},
            {'label': _('Bäckereien'), 'value': '10'},
            {'label': _('Wäschereien'), 'value': '11'},
            {'label': _('Gartenbau'), 'value': '12'},
            {'label': _('Papier und Druck'), 'value': '13'},
            {'label': _('haushaltsähnliche Gewerbebetriebe'), 'value': '14'},
            {'label': _('Summenlastprofil Gewerbe, Handel, Dienstleistung'), 'value': '15'},
        ],
        placeholder=_("Auswahl des Gebäudetyps"),
        id='application',
    ),
        
    dcc.Input(
        id="heatRequirement", type="number",
        placeholder=_("Jahreswärmebedarf in kWh/a"),
        debounce=True, style={'width': '200px', 'height': '25px'}
    ),
    html.Br(),
    dcc.DatePickerRange(
        display_format=' DD/MM/YYYY',
        start_date_placeholder_text=_('Start Datum'),
        end_date_placeholder_text=_('End Datum'),
        id='datePicker'
    ),
    dcc.RadioItems(
        options=[
            {'label': _('Januar'), 'value': '1'},
            {'label': _('Februar'), 'value': '2'},
            {'label': _('März'), 'value': '3'},
            {'label': _('April'), 'value': '4'},
            {'label': _('Mai'), 'value': '5'},
            {'label': _('Juni'), 'value': '6'},
            {'label': _('Juli'), 'value': '7'},
            {'label': _('August'), 'value': '8'},
            {'label': _('Sepember'), 'value': '9'},
            {'label': _('Oktober'), 'value': '10'},
            {'label': _('November'), 'value': '11'},
            {'label': _('Dezember'), 'value': '12'},
            {'label': _('Alle'), 'value': 'All'},
        ],
        value='All',
        id='displayMonth',
        inline=True
    ),
    html.Button(_('Approximation starten'), id='approximationStart'),
    html.Button(_("Download als csv"), id="btn-download-csv"),
    dcc.Download(id="download-csv"),
    dcc.Loading(id="ls-loading",
                children=[html.Div([dcc.Graph(id="heatGraph", figure=go.Figure())])],
                type="circle", fullscreen=False),
    html.P(_('Es gibt keine Eingabe'), id='containerParagraphAtBottom'),
    dcc.Store(id='heat_approximationStoring')
], style={'font-family': "Roboto, sans-serif", "color": "rgb(116, 117, 121)",
          "font-size": "18.75px",
          "font-weight": "400",
          "line-height": "22.5px",
          "overflow-x": "hidden"})
          
@app.callback(
    Output(component_id='datePicker', component_property='start_date'),
    Output(component_id='datePicker', component_property='end_date'),
    Output(component_id='heatRequirement', component_property='value'),
    Output(component_id='application', component_property='value'),
    Output(component_id='displayMonth', component_property='value'),
    Input(component_id='referenceYear', component_property='value'),

    #prevent_initial_call = True,
    #allow_duplicate=True,
)
def resetData(visibility_state):
    
    return None, None, None, None, 'All'

@app.callback(
    Output(component_id = 'hideText', component_property = 'style'),
    Input(component_id = 'referenceYear', component_property = 'value')
   )
def showHideTxt(visibility_state):
    
    if visibility_state == 'off':
        return {'display': 'none'}
    if visibility_state == 'on':
        return {'display': 'block'}

# Hide unnecessary elements for refrenceyear run    
@app.callback(
   Output(component_id = 'hideElements', component_property = 'style'),
   Input(component_id = 'referenceYear', component_property = 'value')
   )
def showHideElement(visibility_state):
    if visibility_state == 'off':
        return {'display': 'block'}
    if visibility_state == 'on':
        return {'display': 'none'}

@app.callback(
    Output('station', 'options'),
    Input('state', 'value'),
    #prevent_initial_call = True
    )

def stationSelection(state:str) -> list:
    return [{"label":row['name'] , "value": row['station_id']}
             
    for index,row in stations.all().df.to_pandas().iterrows() if row['state'] == state]

#Setting of the available date frame
@app.callback(
    Output(component_id = 'datePicker',component_property = 'min_date_allowed'),
    Output(component_id = 'datePicker',component_property = 'max_date_allowed'),
    Input(component_id = 'referenceYear', component_property = 'value'),
    Input(component_id = 'station', component_property = 'value'),
    )
# The following function returns the data range provided by the chosen station
def dateRangePicker(referenceYear: str, stationId: int) -> Tuple[str, str]:
    if referenceYear == "on":
        minDate = datetime.datetime.strptime("01/01/2021", "%m/%d/%Y")
        maxDate = datetime.datetime.strptime("12/31/2021", "%m/%d/%Y")
    else:
        print("stattionID",stationId)
        data = stations.filter_by_station_id(station_id=stationId)
        print("data",data)
        stationData = data.values.all().df#.to_pandas()
        print("stationData",stationData)
        minDate = (min(stationData["date"])).date()
        maxDate = (max(stationData["date"])).date()
        print(minDate, maxDate)
    return minDate, maxDate

#Setting Diplay Month
@app.callback(
    Output('displayMonth','options'),
    Input('datePicker', 'end_date'),
    Input('on-load', 'data'),
    State('datePicker', 'start_date'),
    # prevent_initial_call  = True,
    )
# The following function displays a list of available months in the data range selected
def displayMonths(endDate:str, dataOnLoad, startDate:str)-> list:
    
    if endDate is None or startDate is None:
        return [
                {'label': _('Januar'), 'value': '1'},
                {'label': _('Februar'), 'value': '2'},
                {'label': _('März'), 'value': '3'},
                {'label': _('April'), 'value': '4'},
                {'label': _('Mai'), 'value': '5'},
                {'label': _('Juni'), 'value': '6'},
                {'label': _('Juli'), 'value': '7'},
                {'label': _('August'), 'value': '8'},
                {'label': _('Sepember'), 'value': '9'},
                {'label': _('Oktober'), 'value': '10'},
                {'label': _('November'), 'value': '11'},
                {'label': _('Dezember'), 'value': '12'},
                {'label': _('Alle'), 'value': 'All'},
                ]
    Months = pd.date_range(startDate,
    endDate,freq = 'W').strftime("%B").unique().tolist()
    print(Months)
    # Setting the months in the right format for plotly dash
    displayMonths = [{"label":_(index) , 
        "value": datetime.datetime.strptime(index, "%B").month} 
        for index in Months ]
    print(displayMonths)
    displayMonths.append ({"label":_('Alle') , "value": 'All'})
    print(displayMonths)

    return displayMonths


# Other callbacks need to be similarly adapted.

# Running the app in a Jupyter notebook
# Uncomment the below line when running in a standalone script, not in a Jupyter Notebook.
# app.run_server(debug=True)

#Warme Approximation
@app.callback(
    Output(component_id = 'heatGraph', component_property = 'figure'),
    Output(component_id = 'containerParagraphAtBottom',component_property = 'children'),
    Output(component_id = 'heat_approximationStoring',component_property = 'data'),
    Input('on-load', 'data'),
    Input(component_id = 'approximationStart',component_property = 'n_clicks'),
    Input(component_id = 'displayMonth',component_property = 'value'),
    State(component_id = 'application',component_property = 'value'),
    State(component_id = 'station',component_property = 'value'),
    State(component_id = 'heatRequirement',component_property = 'value'),
    State(component_id = 'datePicker',component_property = 'start_date'),
    State(component_id = 'datePicker',component_property = 'end_date'),
    State(component_id = 'referenceYear', component_property = 'value'),
    prevent_initial_call = True
    )
# This function calculates the approximations and displays it
def updateHeatGraph(onLoadData, n_clicks:int,displayMonth:str,application:str,StationId:int,heatRequirement:int,
                    startDate:str,endDate:str,referenceYear:str):
    #logging.debug(f"Received input value: {onLoadData}")
    if n_clicks == 0 or n_clicks is None:
        fig = go.Figure()
        return fig,_("Es gibt keine Eingabe"),pd.DataFrame.to_dict(pd.DataFrame())

    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0] 
    # Restructure Option 1: Seperate Graph update and calculation
    # Restrucute Option 2: validate, 
    print(int(application),
        heatRequirement,
        StationId,
        startDate,
        endDate,
        referenceYear)
    

    heat = heatLoad(
        int(application),
        heatRequirement,
        StationId,
        startDate,
        endDate,
        referenceYear,
    )
    print("HEAT:",heat[0],heat[1])
   
    heat =  heatLoad(int(application),heatRequirement,StationId,startDate,endDate,referenceYear)
    #global heat_approximation
    heatApproximation = heat[1]
    missingValues = heat[0]
    if displayMonth == 'All':
        result = heatApproximation
    else:
        result=pd.DataFrame({'Last':(heatApproximation.groupby
        (heatApproximation.Time.dt.month).get_group(int(displayMonth)))['Last'],
        'WW_Last':(heatApproximation.groupby(heatApproximation.Time.dt.month).
        get_group(int(displayMonth)))['WW_Last'],
        'Time':(heatApproximation.groupby(heatApproximation.Time.dt.month).
        get_group(int(displayMonth)))
        ['Time'],'fehlend':heatApproximation['fehlend']})
        print(heatApproximation.groupby(heatApproximation.Time.dt.month).head())

    
    fig=make_subplots(specs = [[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(name = _('Wärmelastgang in kW'),x = result['Time'],
    y = result['Last'],mode = 'lines', line = dict(color = "#0000ff")),secondary_y = False)
    fig.add_trace(go.Scatter(name = _('Fehlende Angaben'),x = result['Time'],
    y = result['Last'].where(result['fehlend'] == 'True'),mode = 'lines',
    line = dict(color = "red")),secondary_y = False)
    fig.add_trace(go.Scatter(name=_('Trinkwarmwasser-Lastgang in kW'),
    x = result['Time'],y = result['WW_Last'],mode = 'lines',
    line = dict(color="#f700ff")),secondary_y = True)

    fig.update_xaxes(
    tickangle = 90,
    title_text = _("Datum"),
    title_font = {"size": 20}
    )

    fig.update_yaxes(
    title_text = _("Wärmelastgang in kW"),
    title_standoff = 25
    )

    fig.update_yaxes(
    title_text = _("Trinkwarmwasser-Lastgang in kW"), 
    secondary_y = True
    )
    #fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2], mode='lines')])
    #fig.show()
    print(heatApproximation.head())  # 檢查數據是否正常
    print(f"n_clicks: {n_clicks}, displayMonth: {displayMonth}, application: {application}")
    print(pd.DataFrame.to_dict(heatApproximation))

    return fig, _('Für die ausgewählte Station gibt es ') + str(missingValues) + \
        _(' fehlende Werte, die in der Grafik rot markiert sind. ') ,\
        pd.DataFrame.to_dict(heatApproximation)

# The download csv Funcionality
@app.callback(
    Output("download-csv", "data"),
    Input("btn-download-csv", "n_clicks"),
    Input('heat_approximationStoring','data'),
    Input('application','value'),
    Input('heatRequirement','value'),
    Input('datePicker', 'start_date'),
    Input('datePicker', 'end_date'),
    Input('state', 'value'),
    Input('station','value'),
    Input('referenceYear', 'value'),
    State('station',"options"),
    State("application","options"), 
    prevent_initial_call = True,
)
def downloadAsCsv(nClicks,jsonifiedHeatApproximation:pd.DataFrame,
    application:str,heatRequirement:int,startDate:str,endDate:str,
    state:str,station:str,referenceYear:str, labelsStation,labelsApplication, ):
    #To Do Add download for Wärmelast 
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]      
    if 'btn-download-csv' in changed_id:
            if referenceYear == "off":
                heatApproximation = pd.DataFrame.from_dict(jsonifiedHeatApproximation)
                labelStation = [x['label'] for x in labelsStation if x['value'] == station]
                labelsApplication = [x['label'] for x in labelsApplication 
                if x['value'] == application]
                heatApproximation.columns = [['Jahreswärmebedarfs in kWh/a :'
                +str(heatRequirement),'','',''  ],['Anwendung:'+labelsApplication[0],'','',''],
                ['Zeitraum : Von '+startDate+' Bis '+endDate,'','',''],
                ['Bundesland:'+state + ' Station:' + labelStation[0] ,'','',''],
                ['','','',''],['Datum','WärmeLast in kW','TrinkwasserWärmeLast in kW',
                'FehlendeWerte(Angepasst)']]    
            if referenceYear == "on":
                heatApproximation = pd.DataFrame.from_dict(jsonifiedHeatApproximation)  
            return dcc.send_data_frame(heatApproximation.to_csv,'WarmeData.csv',
                                       index=False,encoding = 'utf-8')
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components

@app.callback(
    Output("headingApp", "children"),
    Output("container1", "children"),
    Output("container2", "children"),
    Output("container3", "children"),
    Output("container4", "children"),
    Output("container5", "children"),
    Output("container6", "children"),
    Output("container7", "children"),
    Output("container8", "children"),
    Output("container9", "children"),
    Output("referenceYear", "options"),
    Output("referenceYear", "placeholder"),
    Output("station", "placeholder"),
    Output('application', 'options'),
    Output('application', 'placeholder'),
    Output("heatRequirement", "placeholder"),
    Output('datePicker', "start_date_placeholder_text"),
    Output('datePicker', "end_date_placeholder_text"),
    Output("approximationStart", "children"),
    Output("btn-download-csv", "children"),
    Input('on-load', 'data'),
    allow_duplicate=True,
)
def update_layout(data):
    headingAppTranslation = _("Wärmelast Approximation")
    container1Translation = _("Als Testrefenzjahr haben wir die folgenden Werte gewählt:")
    container2Translation = _("Koordinatensystem : Lambert konform konisch")
    container3Translation = _("Rechtswert        : 4201500 Meter")
    container4Translation = _("Hochwert          : 2848500 Meter")
    container5Translation = _("Höhenlage        : 36 Meter über NN")
    container6Translation = _("Erstellung des Datensatzes im Mai 2016")
    container7Translation = _("Art des TRY       : mittleres Jahr")
    container8Translation = _("Bezugszeitraum    : 1995-2012")
    container9Translation = _("Datenbasis        : Beobachtungsdaten Zeitraum 1995-2012")
    optionsReferenceYear = [
        {'label': _('Testreferenzjahr'), 'value': 'on'},
        {'label': _('Wetterstation'), 'value': 'off'},     
    ]
    placeholderReferenceYear = _("Berechnungstyp")
    placeholderStationPlaceholder = _("Auswahl der Station")
    optionsDropdown = [
        {'label': _('Einfamilienhaus'), 'value': '2'},
        {'label': _('Mehrfamilienhaus'), 'value': '3'},
        {'label': _('Gebietskörperschaft'), 'value': '4'},
        {'label': _('Einzelhandel, Großhandel'), 'value': '5'},
        {'label': _('Metall, Kfz'), 'value': '6'},
        {'label': _('sonst. betr. Dienstleistungen'), 'value': '7'},
        {'label': _('Gaststätten'), 'value': '8'},
        {'label': _('Beherbergung'), 'value': '9'},
        {'label': _('Bäckereien'), 'value': '10'},
        {'label': _('Wäschereien'), 'value': '11'},
        {'label': _('Gartenbau'), 'value': '12'},
        {'label': _('Papier und Druck'), 'value': '13'},
        {'label': _('haushaltsähnliche Gewerbebetriebe'), 'value': '14'},
        {'label': _('Summenlastprofil Gewerbe, Handel, Dienstleistung'), 'value': '15'},
    ]
    placeholderBuildingType = _("Auswahl des Gebäudetyps")
    heatRequirementPlaceholder = _("Jahreswärmebedarf in kWh/a")
    startDatePlaceholderText = _('Start Datum')
    endDatePlaceholderText = _('End Datum')

    paragraphNoEntry = _('Es gibt keine Eingabe')

    buttonLabelApproximationStart = _('Approximation starten')
    buttonLabelDownloadCsv = _('Download als csv')
    stationplaceholder = _("Auswahl der Station")

    return (
        headingAppTranslation,
        container1Translation,
        container2Translation,
        container3Translation,
        container4Translation,
        container5Translation,
        container6Translation,
        container7Translation,
        container8Translation,
        container9Translation,
        optionsReferenceYear,
        placeholderReferenceYear,
        placeholderStationPlaceholder,
        optionsDropdown,
        placeholderBuildingType,
        heatRequirementPlaceholder,
        startDatePlaceholderText,
        endDatePlaceholderText,
        buttonLabelApproximationStart,
        buttonLabelDownloadCsv,
    )


INFO:wetterdienst.settings:Wetterdienst cache is enabled [CACHE_DIR: /home/trinker/.cache/wetterdienst]
INFO:wetterdienst.core.timeseries.request:Processing request DwdObservationRequest(parameter=[(tt_tu/air_temperature)], resolution=hourly, period=[recent], start_date=None, end_date=None, humanize=True, format=long, si_units=True)
INFO:wetterdienst.provider.dwd.observation.metaindex:Downloading file https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate//hourly/air_temperature/recent/TU_Stundenwerte_Beschreibung_Stationen.txt.


['Frei']


In [9]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
    

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8050
DEBUG:urllib3.connectionpool:http://127.0.0.1:8050 "GET /_alive_79743487-48e7-4d63-879c-90e878880c84 HTTP/1.1" 200 5


INFO:wetterdienst.util.network:Creating dircache folder at /home/trinker/.cache/wetterdienst/43200.0
INFO:wetterdienst.provider.dwd.observation.metaindex:Downloading file https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate//hourly/air_temperature/recent/TU_Stundenwerte_Beschreibung_Stationen.txt.


['Januar']
[{'label': 'Januar', 'value': 1}]
[{'label': 'Januar', 'value': 1}, {'label': 'Alle', 'value': 'All'}]


DEBUG:root:Received input value: on


3 12000 None 2021-01-01 2021-01-16 on


/tmp/ipykernel_26886/1239439221.py:53: DtypeWarning:

Columns (2,3,4,5,7,8,9,10,11,12,13,14,15,17,43,69,95,121,147,173) have mixed types. Specify dtype option on import or set low_memory=False.

DEBUG:root:Received input value: on


This ist the end 2 2021-01-01 2021-01-16
HEAT: 0                          Time      Last   WW_Last fehlend
0   2021-01-01 01:00:00+00:00  1.439670  0.099694   False
1   2021-01-01 02:00:00+00:00  1.473762  0.103081   False
2   2021-01-01 03:00:00+00:00  1.485794  0.104975   False
3   2021-01-01 04:00:00+00:00  1.591058  0.113538   False
4   2021-01-01 05:00:00+00:00  1.907399  0.137399   False
..                        ...       ...       ...     ...
377 2021-01-16 18:00:00+00:00  2.030395  0.189643   False
378 2021-01-16 19:00:00+00:00  2.069549  0.196233   False
379 2021-01-16 20:00:00+00:00  2.064888  0.198686   False
380 2021-01-16 21:00:00+00:00  2.020906  0.197114   False
381 2021-01-16 22:00:00+00:00  1.852828  0.183170   False

[382 rows x 4 columns]


/tmp/ipykernel_26886/1239439221.py:53: DtypeWarning:

Columns (2,3,4,5,7,8,9,10,11,12,13,14,15,17,43,69,95,121,147,173) have mixed types. Specify dtype option on import or set low_memory=False.



This ist the end 2 2021-01-01 2021-01-16
                       Time      Last   WW_Last fehlend
0 2021-01-01 01:00:00+00:00  1.439670  0.099694   False
1 2021-01-01 02:00:00+00:00  1.473762  0.103081   False
2 2021-01-01 03:00:00+00:00  1.485794  0.104975   False
3 2021-01-01 04:00:00+00:00  1.591058  0.113538   False
4 2021-01-01 05:00:00+00:00  1.907399  0.137399   False
n_clicks: 1, displayMonth: All, application: 3
{'Time': {0: Timestamp('2021-01-01 01:00:00+0000', tz='UTC'), 1: Timestamp('2021-01-01 02:00:00+0000', tz='UTC'), 2: Timestamp('2021-01-01 03:00:00+0000', tz='UTC'), 3: Timestamp('2021-01-01 04:00:00+0000', tz='UTC'), 4: Timestamp('2021-01-01 05:00:00+0000', tz='UTC'), 5: Timestamp('2021-01-01 06:00:00+0000', tz='UTC'), 6: Timestamp('2021-01-01 07:00:00+0000', tz='UTC'), 7: Timestamp('2021-01-01 08:00:00+0000', tz='UTC'), 8: Timestamp('2021-01-01 09:00:00+0000', tz='UTC'), 9: Timestamp('2021-01-01 10:00:00+0000', tz='UTC'), 10: Timestamp('2021-01-01 11:00:00+0000', tz='